In [12]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define your Area of Interest (AOI) - Example: part of Delhi
aoi = ee.Geometry.Rectangle([77.5, 28.4, 77.7, 28.6])

# Function to filter and select Sentinel-2 image
def get_sentinel_image(date_start, date_end, max_cloud=20):
    collection = (ee.ImageCollection("COPERNICUS/S2_SR")
                  .filterBounds(aoi)
                  .filterDate(date_start, date_end)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud))
                  .sort('CLOUDY_PIXEL_PERCENTAGE'))
    image = collection.first()
    return image

# Get images with adjusted parameters
image1 = get_sentinel_image('2020-01-01', '2020-01-31')
image2 = get_sentinel_image('2023-12-01', '2024-02-15')  # expanded range for 2024

# Check if images were found
print("Image 1 found:", image1 is not None)
print("Image 2 found:", image2 is not None)

# Select RGB + NIR bands
bands = ['B4', 'B3', 'B2', 'B8']  # Red, Green, Blue, NIR
image1 = image1.select(bands).clip(aoi)
image2 = image2.select(bands).clip(aoi)

# Export to Google Drive
task1 = ee.batch.Export.image.toDrive(
    image=image1,
    description='Sentinel2_Image_2020',
    folder='EarthEngine',
    fileNamePrefix='sentinel2_2020',
    region=aoi,
    scale=10,
    maxPixels=1e9
)

task2 = ee.batch.Export.image.toDrive(
    image=image2,
    description='Sentinel2_Image_2024',
    folder='EarthEngine',
    fileNamePrefix='sentinel2_2024',
    region=aoi,
    scale=10,
    maxPixels=1e9
)

# Start tasks
task1.start()
task2.start()

print("Export tasks started. Check your Google Drive in the 'EarthEngine' folder.")

Image 1 found: True
Image 2 found: True
Export tasks started. Check your Google Drive in the 'EarthEngine' folder.
